<a href="https://colab.research.google.com/github/tsanifaishal/MachineLearningTasks/blob/main/UTS/Decision_Tree_UTS_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : M Tsani Faishal Azhar

NIM : 1103210143

Judul : Tugas UTS Machine learning model Decicion Tree dari data set Credit score classification

In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [152]:
# Memasukan path data train dan data test
pathTrain = "/content/drive/MyDrive/Dataset/train.csv"
pathTest = "/content/drive/MyDrive/Dataset/test.csv"

# Membaca dataset sebagai dataframe
dataTrain = pd.read_csv(pathTrain)
dataTest = pd.read_csv(pathTest)

# Menampilkan beberapa baris pertama dari dataframe
print("Data Train:")
print(dataTrain.head())
print("Data Test:")
print(dataTest.head())

<ipython-input-152-9ccb66a9c53d>:6: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  dataTrain = pd.read_csv(pathTrain)


Data Train:
       ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utilization

In [153]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

#Pre-processing

In [154]:
# # Mengubah kolom 'Age' menjadi numerik
# dataTrain['Age'] = pd.to_numeric(dataTrain['Age'], errors='coerce')
# dataTest['Age'] = pd.to_numeric(dataTest['Age'], errors='coerce')

# Mengubah semua kolom yang seharusnya berisi nilai numerik menjadi numerik
numeric_columns = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_of_Loan', 'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Monthly_Balance']

for column in numeric_columns:
    dataTrain[column] = pd.to_numeric(dataTrain[column], errors='coerce')
    dataTest[column] = pd.to_numeric(dataTest[column], errors='coerce')

# Mengisi nilai yang hilang pada kolom 'Age' dengan median
median_age = dataTrain['Age'].median()
dataTrain['Age'].fillna(median_age, inplace=True)
dataTest['Age'].fillna(median_age, inplace=True)

# Menghilangkan kolom yang tidak diperlukan
dataTrain.drop(['ID', 'Customer_ID', 'Month', 'Name', 'SSN'], axis=1, inplace=True)
dataTest.drop(['ID', 'Customer_ID', 'Month', 'Name', 'SSN'], axis=1, inplace=True)

# Mengisi nilai yang hilang pada kolom 'Monthly_Inhand_Salary' dengan median
median_salary = dataTrain['Monthly_Inhand_Salary'].median()
dataTrain['Monthly_Inhand_Salary'].fillna(median_salary, inplace=True)
dataTest['Monthly_Inhand_Salary'].fillna(median_salary, inplace=True)

# Mengisi nilai yang hilang pada kolom 'Num_of_Delayed_Payment' dengan 0
dataTrain['Num_of_Delayed_Payment'].fillna(0, inplace=True)
dataTest['Num_of_Delayed_Payment'].fillna(0, inplace=True)

# Mengisi nilai yang hilang pada kolom 'Num_Credit_Inquiries' dengan median
median_inquiries = dataTrain['Num_Credit_Inquiries'].median()
dataTrain['Num_Credit_Inquiries'].fillna(median_inquiries, inplace=True)
dataTest['Num_Credit_Inquiries'].fillna(median_inquiries, inplace=True)


# Mengisi nilai yang hilang pada kolom 'Monthly_Balance' dengan median
# median_balance = dataTrain['Monthly_Balance'].median()
# dataTrain['Monthly_Balance'].fillna(median_balance, inplace=True)
# dataTest['Monthly_Balance'].fillna(median_balance, inplace=True)

# # Mengubah kolom-kolom kategorikal menjadi numerik menggunakan one-hot encoding
# dataTrain = pd.get_dummies(dataTrain)
# dataTest = pd.get_dummies(dataTest)

# Memisahkan fitur (X) dan label (y) pada data train dan test
X_train = dataTrain[['Age', 'Monthly_Inhand_Salary', 'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Total_EMI_per_month']].values
y_train = dataTrain['Occupation']
X_test = dataTest[['Age', 'Monthly_Inhand_Salary', 'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Total_EMI_per_month']].values
y_test = dataTest['Occupation']

In [155]:
X_train[0:5]

array([[  23.        , 1824.84333333,    4.        ,   26.82261962,
          49.57494921],
       [  23.        , 3093.745     ,    4.        ,   31.94496006,
          49.57494921],
       [-500.        , 3093.745     ,    4.        ,   28.60935202,
          49.57494921],
       [  23.        , 3093.745     ,    4.        ,   31.37786187,
          49.57494921],
       [  23.        , 1824.84333333,    4.        ,   24.79734691,
          49.57494921]])

In [156]:
y_train[0:5]

0    Scientist
1    Scientist
2    Scientist
3    Scientist
4    Scientist
Name: Occupation, dtype: object

In [157]:
print("Data Train - Missing Values:\n", dataTrain.isnull().sum())
print("\nData Test - Missing Values:\n", dataTest.isnull().sum())


Data Train - Missing Values:
 Age                             0
Occupation                      0
Annual_Income                6980
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                  4785
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment          0
Changed_Credit_Limit            0
Num_Credit_Inquiries            0
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1209
Credit_Score                    0
dtype: int64

Data Test - Missing Values:
 Age                            0
Occupation                     0
Annual_Income               3520
Monthly_Inhand_Salary          0
Num_Bank_Acco

In [158]:
# Membangun model Decision Tree Classifier
model = DecisionTreeClassifier(criterion="gini", max_depth = 10)
model.fit(X_train, y_train)

# Menguji model pada data test
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.12394
